<a href="https://colab.research.google.com/github/tvpian/Foundation_Models/blob/main/GenAI_GAN_Advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Advanced GAN

In [3]:
# Importing the Libraries
import torch , torchvision, os, PIL, pdb
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
from tqdm.auto import tqdm
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [4]:
def show(tensor, num = 25, wamb = 0, name = ''):
  data = tensor.detach().cpu()
  grid = make_grid(data[:num], nrow = 5).permute(1,2,0)
  ##optional
  if wandb == 1 and wanbact == 1:
    wandb.log({name:wandb.Image(grid.numpy().clip(0,1))})
  plt.show(grid.clip(0,1))
  plt.show()


## hypermeters and general parameters
n_epochs = 10000
batch_size = 128
lr = 1e-4
z_dim = 200
device = 'cuda'

cur_step = 0
crit_cycles = []
gen_losses = []
crit_losses = []
show_step = 35
save_step = 35


wandbact = 1




In [9]:
### Libraries to communicate with WANDB

!pip install wandb -qqq
import wandb

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [10]:
wandb.login(key="c47eb3707f8ec450c0120e554ba97982c29cfa53")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [11]:
%%capture
experiment_name = wandb.util.generate_id()

myrun = wandb.init(
    project = "tvp_wgan",
    group = experiment_name,
    config = {
        "optimizer":"adam",
        "model":"wgan gp",
        "epoch":"1000",
        "batch_size":128
    }
)
config=wandb.config



In [13]:
print(experiment_name)


ipc8ggop


In [16]:
# Generator model
class Generator(nn.Module):
  def __init__(self, z_dim = 64, d_dim = 16):
    super(Generator, self).__init__()
    self.z_dim = z_dim
    self.gen = nn.Sequential(
        # ConvTranspose2d: in_channels, out_channels, kernel_size, stride = 1, padding = 0
        # Calculating new width and height: (n-1)*stride - 2*padding + ks
        # n - width or height
        # ks - kernel size
        # input -  1x1 image with z_dim number of channels(200)
        nn.ConvTranspose2d(z_dim, d_dim * 32, 4, 1, 0),  ## 4x4 (ch:200, 512)
        nn.BatchNorm2d(d_dim * 32),
        nn.ReLU(True),

        nn.ConvTranspose2d(d_dim*32, d_dim*16, 4, 2, 1), ## 8x8 (ch:512, 256)
        nn.BatchNorm2d(d_dim * 32),
        nn.ReLU(True),

        nn.ConvTranspose2d(d_dim*16, d_dim*8, 4, 2, 1), ## 16x16 (ch:256, 128)
        nn.BatchNorm2d(d_dim * 16),
        nn.ReLU(True),

        nn.ConvTranspose2d(d_dim*8, d_dim*4, 4, 2, 1), ## 32x32 (ch:128, 64)
        nn.BatchNorm2d(d_dim * 8),
        nn.ReLU(True),

        nn.ConvTranspose2d(d_dim*4, d_dim*2, 4, 2, 1), ## 64x64 (ch:64, 32)
        nn.BatchNorm2d(d_dim * 4),
        nn.ReLU(True),

        nn.ConvTranspose2d(d_dim*2, 3, 4, 2, 1), ## 128x128 (ch:32, 3)
        nn.Tanh() ## Get the results in the range of -1 and 1 preferred for Generative models
    )

  def forward(self, noise):
    x = noise.view() # 128 x 200 x 1 x 1
    return self.gen(x)

  def gen_noise(num, z_dim, device = "cuda"):
    return torch.rand(num, z_dim, device = "cuda") # 128 x 200


In [ ]:
## Critic model
# conv2d: in_channel, out_channels, kernel_size, stride = 1, padding = 0
## New_width and height: # (n+2*pad-ks)//stride + 1

class Critic(nn.Module):
  def __init__(self, d_dim=16):
    super(Critic, self).__init__()

    self.crit = nn.Sequential(
        # Conv2d: in_channels, out_channels, kernel_size, stride = 1, padding = 0
        ## New width and height: # (n+2*pad-ks)//stride + 1
        nn.Conv2d(3, d_dim, 4, 2, 1), #(n+2*pad-ks)//stride + 1 - (128+2*1-4)//2+1 = 64 x 64 (ch: 3, 16)
        nn.InstanceNorm2d(d_dim)
        nn.R
    )